# DESeq2: Differential Expression in the GenePattern Environment

Computational notebooks are powerful tools for data analysis. However, they are best used to run relatively short analyses in an interactive format. This notebook demonstrates how GenePattern modules can be integrated into the notebook environment to integrate longer analyses on larger datasets with more interactive characterization of the results. 

In this notebook, we will identify differentially expressed genes between cancerous and healthy breast tissue using the DESeq2 algorithm [1]. DESeq2 performs a normalization on raw RNA-seq counts, and then models expression using the negative binomial distribution to identify genes that are significantly differentiall expressed in one class of samples compared to another. Here, the normal breast tissue samples comprise one class, and the other is made up of the tumor samples.

# Obtaining data

First, we will obtain the data for this analysis. The data is stored [here](https://datasets.genepattern.org/?prefix=data/module_support_files/DESeq2/). Specifically, we will need to download `BRCA_tumor_and_normal_20783x40.gct`, which contains the expression data, and `BRCA_tumor_and_normal_x40.cls`, which labels which samples are tumor and which are normal. `GCT` and `CLS` are formats used throughout the GenePattern ecosystem to store expression data and corresponding sample-level annotations. You can read about them in the [GenePattern file format guide](https://www.genepattern.org/file-formats-guide).

Using the `!` operator, we can run bash commands directly in the code cell. The follow cells use `wget` to retrieve the files from the link above.

## Download the expression data

In [4]:
!wget https://datasets.genepattern.org/data/module_support_files/DESeq2/BRCA_tumor_and_normal_20783x40.gct

--2020-09-23 23:05:03--  https://datasets.genepattern.org/data/module_support_files/DESeq2/BRCA_tumor_and_normal_20783x40.gct
Resolving datasets.genepattern.org (datasets.genepattern.org)... 52.85.144.128, 52.85.144.36, 52.85.144.59, ...
Connecting to datasets.genepattern.org (datasets.genepattern.org)|52.85.144.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4229844 (4.0M) [binary/octet-stream]
Saving to: ‘BRCA_tumor_and_normal_20783x40.gct.2’

BRCA_tumor_and_norm 100%[===================>]   4.03M  --.-KB/s    in 0.03s   

2020-09-23 23:05:03 (131 MB/s) - ‘BRCA_tumor_and_normal_20783x40.gct.2’ saved [4229844/4229844]



## Download the class file

In [5]:
!wget https://datasets.genepattern.org/data/module_support_files/DESeq2/BRCA_tumor_and_normal_x40.cls

--2020-09-23 23:05:04--  https://datasets.genepattern.org/data/module_support_files/DESeq2/BRCA_tumor_and_normal_x40.cls
Resolving datasets.genepattern.org (datasets.genepattern.org)... 52.85.144.66, 52.85.144.128, 52.85.144.36, ...
Connecting to datasets.genepattern.org (datasets.genepattern.org)|52.85.144.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118 [binary/octet-stream]
Saving to: ‘BRCA_tumor_and_normal_x40.cls.2’

BRCA_tumor_and_norm 100%[===================>]     118  --.-KB/s    in 0s      

2020-09-23 23:05:04 (4.17 MB/s) - ‘BRCA_tumor_and_normal_x40.cls.2’ saved [118/118]



# Uploading data to the GenePattern server

After running the two cells above, you'll notice that `BRCA_tumor_and_normal_20783x40.gct` and `BRCA_tumor_and_normal_x40.cls` are now in the same directory as this notebook in your GenePattern Notebook directory. Before we can use these files in a GenePattern module, we need to upload them to the GenePattern server. We can do this using GenePattern's REST API. The following cell will return a `GPServer` object that is authorized to send and receive files from your account on `cloud.genepattern.org`. You will need to edit it to contain your username.

## Login to GenePattern for REST API access

In [6]:
from getpass import getpass
import gp

username = input("Username: ")

passwd = getpass("Password: ")
gpserver = gp.GPServer('https://cloud.genepattern.org/gp', username, passwd)
gpserver.login()

print("Login successful!")

Username: atwenzel2
Password: ········
Login successful!


Now we will upload the GCT and CLS file, and importantly, print out the URL, which we will use as the input to DESeq2.

## Upload GCT file

In [7]:
gpserver.upload_file(
    "BRCA_tumor_and_normal_20783x40.gct", 
    "BRCA_tumor_and_normal_20783x40.gct"
).get_url()

'https://cloud.genepattern.org/gp/users/atwenzel2/tmp/run8178161089531844216.tmp/BRCA_tumor_and_normal_20783x40.gct'

## Upload CLS file

In [8]:
gpserver.upload_file(
    "BRCA_tumor_and_normal_x40.cls", 
    "BRCA_tumor_and_normal_x40.cls"
).get_url()

'https://cloud.genepattern.org/gp/users/atwenzel2/tmp/run5772468078584039820.tmp/BRCA_tumor_and_normal_x40.cls'

# Run DESeq2

With the data uploaded to the GenePattern server, we can now run DESeq2. For the sake of time, this notebook contains a pre-computed job result, but feel free to experiment with running the module later on. 

## Login to the GenePattern server

In [10]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

## Load DESeq2 and run

Here, you can set the `Input file` to be the link to the `gct` file printed above, and the `cls file` to be the link to the `cls` file above. Leave all other inputs blank or as their defaults.

In [11]:
deseq2_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00362')
deseq2_job_spec = deseq2_task.make_job_spec()
deseq2_job_spec.set_parameter("input.file", "https://cloud.genepattern.org/gp/users/atwenzel2/tmp/run8178161089531844216.tmp/BRCA_tumor_and_normal_20783x40.gct")
deseq2_job_spec.set_parameter("cls.file", "https://cloud.genepattern.org/gp/users/atwenzel2/tmp/run5772468078584039820.tmp/BRCA_tumor_and_normal_x40.cls")
deseq2_job_spec.set_parameter("confounding.variable.cls.file", "")
deseq2_job_spec.set_parameter("output.file.base", "<input.file_basename>")
deseq2_job_spec.set_parameter("qc.plot.format", "skip")
deseq2_job_spec.set_parameter("fdr.threshold", "0.1")
deseq2_job_spec.set_parameter("top.N.count", "20")
deseq2_job_spec.set_parameter("random.seed", "779948241")
deseq2_job_spec.set_parameter("job.memory", "2 Gb")
deseq2_job_spec.set_parameter("job.queue", "gp-cloud-default")
deseq2_job_spec.set_parameter("job.cpuCount", "1")
deseq2_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(deseq2_task)


job279004 = gp.GPJob(genepattern.session.get(0), 279004)
genepattern.display(job279004)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00362')

In [12]:
# More documentation can be obtained at the GenePattern website, or by calling help(job279004).
brca_tumor_and_normal_20783x40_matched_normal_vs_primary_tumor_deseq2_results_report_txt_279004 = job279004.get_file("BRCA_tumor_and_normal_20783x40.matched_normal.vs.primary_tumor.DESeq2_results_report.txt")
brca_tumor_and_normal_20783x40_matched_normal_vs_primary_tumor_deseq2_results_report_txt_279004 

# Visualize results

Now that DESeq2 is complete, we can visualize the results.

Help on GPJob in module gp.core object:

class GPJob(GPResource)
 |  GPJob(server_data, uri)
 |  
 |  A running or completed job on a Gene Pattern server.
 |  
 |  Contains methods to get the info of the job, and to wait on a running job by
 |  polling the server until the job is completed.
 |  
 |  Method resolution order:
 |      GPJob
 |      GPResource
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, server_data, uri)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_child_jobs(self)
 |      Queries the GenePattern server for child jobs of this job, creates GPJob
 |      objects representing each of them and assigns the list of them to the
 |      GPJob.children property. Then return this list.
 |  
 |  get_comments(self)
 |      Returns the comments for the job, querying the
 |      server if necessary.
 |  
 |  get_file(self, name)
 |      Returns the output file with the specified name, if no output files
 |      ma

# References

1. Love MI, Huber W, Anders S (2014). “Moderated estimation of fold change and dispersion for RNA-seq data with DESeq2.” Genome Biology, 15, 550. doi: 10.1186/s13059-014-0550-8.